In [1]:
import pandas as pd

import sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split

from feature_engineering import feature_engineering
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from feature_builder import process_dataset

[nltk_data] Downloading package stopwords to /home/andres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
tweets = pd.read_csv('train.csv')

In [3]:
dataset = feature_engineering(tweets)
# Me queda ver como juntar este set con el de TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x_tfidf = v.fit_transform(tweets['text'])


In [5]:
y = tweets.loc[:,'target']

In [32]:
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(x_tfidf, y, test_size = .33, random_state = 17)

In [33]:
clf = svm.SVC()
clf.fit(x_train_tfidf, y_train_tfidf)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [34]:
y_pred_tfidf = clf.predict(x_test_tfidf)
accuracy_score(y_test_tfidf, y_pred_tfidf)

0.8010346199761241

In [6]:
x_embedd = process_dataset(tweets)

In [7]:
x_embedd = x_embedd.iloc[:,:299]

Me quede solo con los embeddings ya que el resto deteriora el algoritmo por alguna razón.

In [9]:
x_train_embedd, x_test_embedd, y_train_embedd, y_test_embedd = train_test_split(x_embedd, y, test_size = .33, random_state = 17)

In [10]:
clf = svm.SVC()
clf.fit(x_train_embedd, y_train_embedd)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
y_pred_embedd = clf.predict(x_test_embedd)
accuracy_score(y_test_embedd, y_pred_embedd)

0.8241146040588938